# 01 — QA с цитатами (RAG шаблон для олимпиадных задач)

Этот ноутбук — универсальный шаблон для задач формата «вопрос → ответ + цитаты» с использованием RAG. Шаги пайплайна:
1. Считываем корпус и вопросы.
2. Строим индексы (BM25 и/или dense).
3. Ретривим кандидаты, при желании реранкаем и делаем MMR.
4. Собираем контекст и формируем промпт.
5. Вызываем LLM, парсим JSON-ответ с цитатами.
6. Считаем метрики, готовим сабмит.

Все места, которые надо адаптировать под конкретное соревнование, помечены `TODO` и снабжены пояснениями.

## 1. Подключение данных задачи

### 1.1 Понимание формата задачи

Чек-лист перед стартом:
- Какие колонки есть в train/val/test: `id`/`qid`, `question`, `answer` (gold) и т.д.
- Где лежит корпус: один файл или директория с множеством документов.
- Какой формат сабмита: одна колонка `answer`, JSON-строка с полями или несколько колонок. Отредактируйте ниже соответствующие шаги.

In [ ]:

# Пути к данным. Подставьте реальные пути под свою задачу.
DATA_DIR = "data"  # TODO: поменяйте, если данные лежат в другом месте
TRAIN_PATH = f"{DATA_DIR}/train.csv"  # TODO: поставьте путь к train (csv/json/tsv)
VAL_PATH = f"{DATA_DIR}/val.csv"      # TODO: если вал в отдельном файле
TEST_PATH = f"{DATA_DIR}/test.csv"    # TODO: путь к тесту
CORPUS_PATH = f"{DATA_DIR}/corpus"    # TODO: директория или файл с корпусом
OUTPUT_DIR = "outputs"                # TODO: куда сохранять предсказания/сабмиты

# Флаги контроля процесса
HAS_VAL = False  # TODO: True, если есть отдельный validation split
HAS_GOLD_ANSWERS = True  # TODO: False, если в train нет правильных ответов
FAST_DEV_RUN = True  # Включите, чтобы прогонять пайплайн только на нескольких примерах для быстрой проверки

# Создадим директорию для выводов, если нужно
os.makedirs(OUTPUT_DIR, exist_ok=True)


In [ ]:

import pandas as pd

# TODO: подстройте под формат своих файлов (csv/json/tsv/parquet)
train_df = pd.read_csv(TRAIN_PATH) if os.path.exists(TRAIN_PATH) else pd.DataFrame()
val_df = pd.read_csv(VAL_PATH) if HAS_VAL and os.path.exists(VAL_PATH) else pd.DataFrame()
test_df = pd.read_csv(TEST_PATH) if os.path.exists(TEST_PATH) else pd.DataFrame()

# TODO: замените имена колонок на реальные из вашего датасета
# Например, если в файле колонка называется "question_text", переименуйте её в "question" ниже.
renames = {
    'id': 'qid',  # TODO: замените, если идентификатор называется иначе
    'question': 'question',  # TODO: замените на реальное имя
    'answer': 'answer',  # TODO: если gold ответы есть в train
}
train_df = train_df.rename(columns={k: v for k, v in renames.items() if k in train_df.columns})
val_df = val_df.rename(columns={k: v for k, v in renames.items() if k in val_df.columns})
test_df = test_df.rename(columns={k: v for k, v in renames.items() if k in test_df.columns})

# Проверим обязательные колонки
required_cols = ['qid', 'question']
missing = [c for c in required_cols if (not train_df.empty and c not in train_df.columns)]
if missing:
    print(f"[WARNING] В train отсутствуют колонки: {missing}. Проверьте rename выше.")


In [ ]:

print("Размеры данных:")
print({
    'train': train_df.shape,
    'val': val_df.shape,
    'test': test_df.shape,
})

if not train_df.empty:
    print()
    print("Первые строки train (проверьте, что колонки правильные):")
    display(train_df.head())
    if 'question' in train_df.columns:
        print()
        print("Пример вопросов:")
        print(train_df['question'].head(3).tolist())
    if 'answer' in train_df.columns:
        print()
        print("Пример ответов:")
        print(train_df['answer'].head(3).tolist())
else:
    print("[INFO] train_df пустой — заполните пути и перезагрузите.")


## 2. Конфигурация RAG

Гиперпараметры, которые чаще всего двигают качество:
- `TOP_K_RETRIEVAL`: сколько чанков брать на первом шаге ретрива (чем больше, тем медленнее, но выше recall).
- `TOP_K_CE`: сколько оставить для cross-encoder; обычно меньше, чем TOP_K_RETRIEVAL.
- `MAX_CONTEXT_TOKENS`: ограничение размера контекста для LLM.
- `ALPHA_HYBRID`: баланс BM25 vs dense в гибридном retriever (0 — только BM25, 1 — только dense).
- Флаги: `USE_BM25`, `USE_DENSE`, `USE_HYBRID`, `USE_CROSS_ENCODER`, `USE_MMR` — включайте/выключайте каналы, когда тестируете разные режимы.

In [ ]:

# Базовые гиперпараметры retriever/LLM
TOP_K_RETRIEVAL = 50  # TODO: уменьшите для скорости, увеличьте для полноты
TOP_K_CE = 20         # TODO: сколько кандидатов отдавать cross-encoder
MAX_CONTEXT_TOKENS = 1024  # TODO: ограничение на суммарные токены контекста
ALPHA_HYBRID = 0.5    # TODO: 0.0 => чистый BM25, 1.0 => чистый dense

USE_BM25 = True       # TODO: выключите, если хотите тестировать только dense
USE_DENSE = True      # TODO: выключите, если нет эмбеддеров
USE_HYBRID = True     # TODO: False, если хотите явно выбрать bm25_candidates или dense_candidates
USE_CROSS_ENCODER = False  # TODO: включите, если есть готовый cross-encoder
USE_MMR = False       # TODO: включите, если хотите диверсифицировать выдачу (требует поддержки в retrieval функциях)

VERBOSE_RAG = True
SHOW_PROGRESS = True


In [ ]:

import json
import random
from typing import Any, List, Optional

import numpy as np
from tqdm.auto import tqdm

from rag.corpus import (
    Corpus,
    build_corpus_from_texts,
    add_chunks_to_corpus,
    get_chunk_texts,
    get_chunk,
    get_document,
)
from rag import chunkers
from rag.indices import build_bm25_index, build_dense_index, BM25Index, DenseIndex
from rag.embeddings import EmbeddingCache
from rag.retrieval import bm25_candidates, dense_candidates, hybrid_candidates
from rag.rerank import rerank_cross_encoder
from rag.context import build_context_window
from rag.debug import inspect_and_print, print_candidates_summary, print_stage_transition
from rag.candidates import Candidate

# Если какие-то импорты пока не нужны, можно закомментировать, но пусть остаются как шпаргалка доступных функций.


In [ ]:

RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
# Фиксируем сиды, чтобы результаты были воспроизводимыми при перезапуске ноутбука.


## 3. Корпус и чанки

### 3.1 Откуда брать корпус
- Корпус может быть заранее собран в одном файле (txt/json) или лежать как набор файлов в директории.
- Под конкретную задачу надо:
  - либо поменять чтение файлов ниже,
  - либо заменить блок на загрузку готового `Corpus` из pickle/JSON (этот шаблон не сохраняет/не грузит на диск, но вы можете добавить).

In [ ]:

# TODO: определите, как устроен корпус. Ниже — простейший пример: каждый файл в директории CORPUS_PATH становится отдельным документом.

def load_raw_corpus(corpus_path: str) -> List[str]:
    texts = []
    if os.path.isdir(corpus_path):
        for fname in sorted(os.listdir(corpus_path)):
            fpath = os.path.join(corpus_path, fname)
            if os.path.isfile(fpath):
                with open(fpath, 'r', encoding='utf-8') as f:
                    texts.append(f.read())
    elif os.path.isfile(corpus_path):
        with open(corpus_path, 'r', encoding='utf-8') as f:
            texts.append(f.read())
    else:
        print(f"[WARNING] Корпус по пути {corpus_path} не найден. Заполните CORPUS_PATH.")
    return texts

raw_texts = load_raw_corpus(CORPUS_PATH)
corpus = build_corpus_from_texts(raw_texts)
print(f"Загружено документов: {len(corpus.documents)}")


In [ ]:

# TODO: выберите тип чанкинга под свой корпус.
# Варианты:
# - chunkers.char_chunker(chunk_size=1000, overlap=200)
# - chunkers.ParagraphChunker() если абзацы уже размечены
# - chunkers.TokenChunker(tokenizer=..., chunk_size=256, overlap=32) если нужен контроль по токенам LLM

chunker = chunkers.char_chunker(chunk_size=1000, overlap=200)
add_chunks_to_corpus(corpus, chunker=chunker)
print(f"После чанкинга чанков: {len(corpus.chunks)}")


In [ ]:

if corpus.chunks:
    avg_len = sum(len(ch.text) for ch in corpus.chunks) / len(corpus.chunks)
    print(f"Средняя длина чанка (символов): {avg_len:.1f}")
    print("Пример первых 3 чанков:")
    for idx, ch in enumerate(corpus.chunks[:3]):
        print(f"[chunk_idx={idx}] doc_id={ch.document_id}\n{ch.text[:400]}\n---")
else:
    print("[WARNING] Чанков нет — проверьте шаг загрузки корпуса и чанкинг.")


## 4. Индексы BM25 и Dense

Зачем два индекса:
- **BM25** — быстро ищет по ключевым словам, устойчив к редким терминам.
- **Dense** — семантический поиск, работает на эмбеддингах, может требовать GPU.
- **Гибрид** совмещает преимущества: BM25 для точных совпадений, dense для перефразов.

In [ ]:

chunk_texts = get_chunk_texts(corpus)
chunk_meta = []
for idx, ch in enumerate(corpus.chunks):
    chunk_meta.append({
        'doc_id': ch.document_id,
        'chunk_idx': idx,
    })
print(f"Подготовлено {len(chunk_texts)} текстов для индексации.")


In [ ]:

bm25_index = None
if USE_BM25:
    bm25_index = build_bm25_index(
        docs=chunk_texts,
        meta=chunk_meta,
        verbose=VERBOSE_RAG,
        show_progress=SHOW_PROGRESS,
    )
    # TODO: при желании сохраните bm25_index на диск и подгружайте при следующих запусках.
else:
    print("[INFO] BM25 выключен.")


In [ ]:

dense_index = None
emb_cache = None
if USE_DENSE:
    # TODO: инициализируйте модель эмбеддингов. Пример для SentenceTransformers:
    # from sentence_transformers import SentenceTransformer
    # emb_model = SentenceTransformer("your-emb-model-id")
    emb_model = None  # TODO: замените на реальную модель с методом .encode
    emb_cache = EmbeddingCache(max_size=10_000)
    dense_index = build_dense_index(
        docs=chunk_texts,
        emb_model=emb_model,
        cache=emb_cache,
        model_id="your-emb-model-id",  # TODO: укажите фактический id модели
        verbose=VERBOSE_RAG,
        show_progress=SHOW_PROGRESS,
    )
    # TODO: как и BM25, dense_index можно кэшировать на диск.
else:
    print("[INFO] Dense индекс выключен.")


In [ ]:

if bm25_index and dense_index:
    assert bm25_index.n_docs == dense_index.n_docs == len(corpus.chunks), "Несогласованное число документов в индексах"
    print(f"Размерность эмбеддингов dense: {dense_index.dim}")
elif bm25_index:
    assert bm25_index.n_docs == len(corpus.chunks)
elif dense_index:
    assert dense_index.n_docs == len(corpus.chunks)
else:
    print("[WARNING] Оба индекса выключены — RAG работать не будет.")


## 5. LLM-клиент и промпт

Модель должна отвечать **строго в JSON**:
```json
{
  "answer": "Краткий текстовый ответ",
  "citations": [12, 47]
}
```
В промпт передаются:
- вопрос,
- несколько чанков с их `chunk_idx` и текстом,
- инструкция отвечать коротко и на основе контекста.

In [ ]:

LLM_MODEL_NAME = "local-llm"  # TODO: укажите модель или endpoint
LLM_MAX_TOKENS = 256          # TODO: ограничение длины ответа
LLM_TEMPERATURE = 0.0         # TODO: температуру можно поднять для более креативных ответов


In [ ]:

def build_prompt(question: str, context_chunks: List[dict], extra_instructions: str = "") -> str:
    """
    Собирает промпт для LLM:
    - формулировка вопроса;
    - список чанков с их индексами и текстом;
    - инструкция отвечать строго в JSON-формате {"answer": ..., "citations": [...]}.

    TODO: адаптируйте тон инструкции под язык и требования конкретной задачи.
    """
    context_str = "\n\n".join(
        [f"[chunk_idx={c['chunk_idx']}]\n{c['text']}" for c in context_chunks]
    )
    instructions = (
        "Ответь на вопрос кратко, используя только факты из контекста. "
        "Формат: JSON с полями 'answer' и 'citations' (список chunk_idx). "
        f"{extra_instructions}"
    )
    prompt = (
        "Ты — помощник, который отвечает строго на основе приведённых фрагментов.\n"
        f"Вопрос: {question}\n\n"
        f"Контекст:\n{context_str}\n\n"
        f"Инструкция: {instructions}\n"
    )
    return prompt


In [ ]:

def call_llm(prompt: str) -> str:
    """
    TODO: реализуйте конкретный вызов LLM.
    Варианты:
    - HTTP-запрос к локальному серверу с моделью.
    - Вызов модели из transformers/Pipeline.
    - Использование готового клиента из инфраструктуры соревнования.

    Функция должна вернуть сырую строку ответа модели.
    """
    raise NotImplementedError("Заполните call_llm под свою среду LLM.")


## 6. Пайплайн для одного вопроса

Шаги:
1. Ретривим кандидаты через `hybrid_candidates` (или отдельно BM25/dense).
2. Опционально реранкаем через `rerank_cross_encoder`.
3. Опционально применяем MMR для диверсификации.
4. Собираем контекст через `build_context_window`.
5. Формируем промпт → вызываем LLM → парсим JSON.
6. Возвращаем ответ, цитаты и отладочную информацию.

In [ ]:

def answer_one_question(
    qid: Any,
    question: str,
    corpus: Corpus,
    bm25_index: Optional[BM25Index],
    dense_index: Optional[DenseIndex],
    emb_model: Any,
    emb_cache: Optional[EmbeddingCache] = None,
    use_hybrid: bool = True,
    use_ce: bool = False,
    use_mmr: bool = False,
) -> dict:
    """
    Выполняет полный шаг RAG для одного вопроса:
    - ретрив (bm25/dense/hybrid)
    - (опционально) cross-encoder
    - (опционально) MMR
    - сбор контекста
    - вызов LLM
    - парсинг ответа и цитат

    Возвращает dict с полями:
    {
      "qid": ...,
      "question": ...,
      "answer": ...,
      "citations": [...],
      "raw_llm_output": ...,
      "retrieval_candidates": candidates_after_all_steps,
    }
    """
    if use_hybrid and bm25_index and dense_index:
        candidates = hybrid_candidates(
            question,
            bm25_index=bm25_index,
            dense_index=dense_index,
            top_k=TOP_K_RETRIEVAL,
            alpha=ALPHA_HYBRID,
            emb_model=emb_model,
            emb_cache=emb_cache,
            verbose=VERBOSE_RAG,
        )
    elif bm25_index and USE_BM25:
        candidates = bm25_candidates(question, bm25_index=bm25_index, top_k=TOP_K_RETRIEVAL, verbose=VERBOSE_RAG)
    elif dense_index and USE_DENSE:
        candidates = dense_candidates(
            question,
            dense_index=dense_index,
            emb_model=emb_model,
            emb_cache=emb_cache,
            top_k=TOP_K_RETRIEVAL,
            verbose=VERBOSE_RAG,
        )
    else:
        raise ValueError("Нет доступных индексов для ретрива. Включите USE_BM25 или USE_DENSE.")

    print_stage_transition("retrieval -> rerank")

    if use_ce and USE_CROSS_ENCODER:
        # TODO: передайте сюда свою модель cross-encoder и токенайзер, если они есть
        candidates = rerank_cross_encoder(
            question,
            candidates,
            model=None,  # TODO: замените на реальную модель
            tokenizer=None,  # TODO: замените на реальный токенайзер
            top_k=TOP_K_CE,
            verbose=VERBOSE_RAG,
        )
    else:
        candidates = candidates[:TOP_K_CE]

    # TODO: если ваш retrieval поддерживает MMR, включите его здесь
    if use_mmr and hasattr(candidates, 'apply_mmr'):
        candidates = candidates.apply_mmr(lambda_=0.5)  # параметр lambda_ подберите под задачу

    print_candidates_summary(candidates, corpus=corpus, limit=5)

    context_chunks = build_context_window(
        question=question,
        candidates=candidates,
        corpus=corpus,
        max_tokens=MAX_CONTEXT_TOKENS,
    )

    prompt = build_prompt(question=question, context_chunks=context_chunks, extra_instructions="")
    raw_output = call_llm(prompt)
    answer, citations = parse_llm_output_to_answer_and_citations(raw_output)

    return {
        "qid": qid,
        "question": question,
        "answer": answer,
        "citations": citations,
        "raw_llm_output": raw_output,
        "retrieval_candidates": candidates,
    }


## 7. Нормализация и парсинг ответа

Для метрик вроде EM/F1 важно нормализовать ответы: привести к нижнему регистру, убрать лишние пробелы и пунктуацию (если это соответствует метрике задачи).

In [ ]:

import re
import string

def normalize_answer(text: str) -> str:
    """
    Базовая нормализация: нижний регистр, убираем пунктуацию и лишние пробелы.
    TODO: адаптируйте под метрику конкретной задачи (например, не убирать пунктуацию, если она важна).
    """
    if text is None:
        return ""
    text = text.lower()
    text = re.sub(f"[{re.escape(string.punctuation)}]", " ", text)
    text = " ".join(text.split())
    return text


In [ ]:

def parse_llm_output_to_answer_and_citations(raw_output: str) -> tuple[str, List[int]]:
    """
    Парсит строку raw_output в (answer, citations).

    Ожидается JSON:
    {
      "answer": "...",
      "citations": [12, 47]
    }

    TODO: добавьте try/except, fallback и более строгую валидацию под задачу.
    """
    answer = ""
    citations: List[int] = []
    try:
        parsed = json.loads(raw_output)
        answer = parsed.get("answer", "") if isinstance(parsed, dict) else ""
        citations = parsed.get("citations", []) if isinstance(parsed, dict) else []
        # Возможен случай, когда citations пришёл как строка или список строк
        if isinstance(citations, str):
            try:
                citations = json.loads(citations)
            except json.JSONDecodeError:
                citations = []
        citations = [int(c) for c in citations if str(c).isdigit()]
    except Exception as e:
        print(f"[WARNING] Не удалось распарсить JSON ответа: {e}. Возвращаем исходную строку.")
        answer = raw_output
        citations = []
    return answer, citations


## 8. Метрики и валидация

Базовые метрики для QA:
- **Exact Match (EM)** по нормализованным строкам.
- **F1** по токенам (precision/recall на уровне слов).

Если у одного вопроса несколько верных ответов, адаптируйте функции ниже (например, берите максимум по кандидату). Если в соревновании есть официальная метрика, лучше скопировать её реализацию сюда вместо базовых EM/F1.

In [ ]:

def compute_em(pred: str, gold: str) -> float:
    return float(normalize_answer(pred) == normalize_answer(gold))


def compute_f1(pred: str, gold: str) -> float:
    pred_tokens = normalize_answer(pred).split()
    gold_tokens = normalize_answer(gold).split()
    common = set(pred_tokens) & set(gold_tokens)
    if not pred_tokens or not gold_tokens:
        return float(pred_tokens == gold_tokens)
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(gold_tokens)
    if precision + recall == 0:
        return 0.0
    return 2 * precision * recall / (precision + recall)


In [ ]:

# TODO: если нет отдельной валидации, можно сделать сплит из train
# from sklearn.model_selection import train_test_split
# if not HAS_VAL and not train_df.empty:
#     train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=RANDOM_SEED)
#     print(f"Создана валидация размером {val_df.shape[0]}")


## 9. Быстрый прогон на валидации

Начните с малого: прогоните пару вопросов, чтобы увидеть, адекватен ли контекст и в правильном ли формате модель возвращает JSON и цитаты.

In [ ]:

N_SMALL = 10
small_val_df = val_df.head(N_SMALL) if not val_df.empty else pd.DataFrame()
small_results = []

if small_val_df.empty:
    print("[INFO] val_df пустой или отсутствует — пропускаем мини-валидацию.")
else:
    for _, row in tqdm(small_val_df.iterrows(), total=len(small_val_df)):
        res = answer_one_question(
            qid=row.get('qid'),
            question=row.get('question'),
            corpus=corpus,
            bm25_index=bm25_index,
            dense_index=dense_index,
            emb_model=None,  # TODO: передайте emb_model, если используете dense/hybrid
            emb_cache=emb_cache,
            use_hybrid=USE_HYBRID,
            use_ce=USE_CROSS_ENCODER,
            use_mmr=USE_MMR,
        )
        res['gold_answer'] = row.get('answer', '')
        small_results.append(res)

    for r in small_results[:3]:
        print("\n=== Пример ===")
        print(f"qid: {r['qid']}")
        print(f"Вопрос: {r['question']}")
        print(f"Ответ модели: {r['answer']}")
        print(f"Цитаты: {r['citations']}")
        if 'gold_answer' in r:
            print(f"Gold: {r['gold_answer']}")
        inspect_and_print(r['retrieval_candidates'], corpus=corpus, limit=3)


In [ ]:

val_predictions = []
if not val_df.empty and HAS_GOLD_ANSWERS:
    for _, row in tqdm(val_df.iterrows(), total=len(val_df)):
        res = answer_one_question(
            qid=row.get('qid'),
            question=row.get('question'),
            corpus=corpus,
            bm25_index=bm25_index,
            dense_index=dense_index,
            emb_model=None,  # TODO: передайте emb_model
            emb_cache=emb_cache,
            use_hybrid=USE_HYBRID,
            use_ce=USE_CROSS_ENCODER,
            use_mmr=USE_MMR,
        )
        pred_answer = res['answer']
        gold_answer = row.get('answer', '')
        res['gold_answer'] = gold_answer
        res['em'] = compute_em(pred_answer, gold_answer)
        res['f1'] = compute_f1(pred_answer, gold_answer)
        val_predictions.append(res)

    val_pred_df = pd.DataFrame(val_predictions)
    print("Средний EM:", val_pred_df['em'].mean())
    print("Средний F1:", val_pred_df['f1'].mean())
    val_pred_path = os.path.join(OUTPUT_DIR, "val_predictions.csv")
    val_pred_df.to_csv(val_pred_path, index=False)
    print(f"Сохранено: {val_pred_path}")
else:
    print("[INFO] Полная валидация пропущена (нет val или gold ответов).")


## 10. Быстрая подстройка гиперпараметров ретрива (опционально)

Что крутить в первую очередь:
- `TOP_K_RETRIEVAL` (recall vs скорость)
- `ALPHA_HYBRID` (баланс BM25/dense)
- `TOP_K_CE` (насколько глубоко реранкать)
- `MAX_CONTEXT_TOKENS` (обрезка контекста)
- включение/выключение MMR и cross-encoder.

In [ ]:

# TODO: небольшой перебор конфигов на подмножестве валидации
val_df_sample = val_df.head(50) if not val_df.empty else pd.DataFrame()
configs = [
    {"TOP_K_RETRIEVAL": 30, "ALPHA_HYBRID": 0.3},
    {"TOP_K_RETRIEVAL": 50, "ALPHA_HYBRID": 0.5},
]

results_grid = []
if val_df_sample.empty or not HAS_GOLD_ANSWERS:
    print("[INFO] Пропускаем гипер-перебор (нет валидации или нет gold ответов).")
else:
    for cfg in configs:
        em_scores, f1_scores = [], []
        for _, row in val_df_sample.iterrows():
            # Временно подменяем глобальные параметры
            TOP_K_RETRIEVAL = cfg["TOP_K_RETRIEVAL"]
            ALPHA_HYBRID = cfg["ALPHA_HYBRID"]
            res = answer_one_question(
                qid=row.get('qid'),
                question=row.get('question'),
                corpus=corpus,
                bm25_index=bm25_index,
                dense_index=dense_index,
                emb_model=None,  # TODO: передайте emb_model
                emb_cache=emb_cache,
                use_hybrid=USE_HYBRID,
                use_ce=USE_CROSS_ENCODER,
                use_mmr=USE_MMR,
            )
            em_scores.append(compute_em(res['answer'], row.get('answer', '')))
            f1_scores.append(compute_f1(res['answer'], row.get('answer', '')))
        results_grid.append({
            'config': cfg,
            'em': np.mean(em_scores),
            'f1': np.mean(f1_scores),
        })
    results_df = pd.DataFrame(results_grid)
    display(results_df)


## 11. Инференс на тесте и сабмит

У `test_df` обычно есть только `qid` и `question`. Формат сабмита берите из условия: этот шаблон создаёт колонку `answer` и опционально `citations_json` (строка JSON со списком ссылок). Отредактируйте под требования соревнования.

In [ ]:

submission_rows = []
if not test_df.empty:
    for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
        res = answer_one_question(
            qid=row.get('qid'),
            question=row.get('question'),
            corpus=corpus,
            bm25_index=bm25_index,
            dense_index=dense_index,
            emb_model=None,  # TODO: передайте emb_model
            emb_cache=emb_cache,
            use_hybrid=USE_HYBRID,
            use_ce=USE_CROSS_ENCODER,
            use_mmr=USE_MMR,
        )
        submission_rows.append({
            'qid': res['qid'],
            'answer': res['answer'],  # TODO: подстройте под нужный формат сабмита
            'citations_json': json.dumps(res['citations']),
        })
else:
    print("[INFO] test_df пустой — проверьте путь к тестовым данным.")


In [ ]:

if submission_rows:
    submission_df = pd.DataFrame(submission_rows)
    submission_path = os.path.join(OUTPUT_DIR, "submission_qa_with_citations.csv")
    submission_df.to_csv(submission_path, index=False)
    print(f"Сабмит сохранён: {submission_path}")
    display(submission_df.head())


## 12. Что сохранить на будущее (опционально)
- Финальный сабмит.
- `val_predictions.csv` для анализа ошибок.
- Конфиг гиперпараметров (можно сохранить в JSON рядом с сабмитом).
- Логи/заметки о том, какие настройки работали лучше всего.